# Save NIDM-Results packs to NiMARE dataset

TODO:
- Create second dataset with local maxima.

In [1]:
import json
from glob import glob
import nibabel as nib
import pandas as pd
import numpy as np
import scipy.ndimage.measurements as meas
from os.path import basename, join, isfile

In [2]:
f2 = '/Users/tsalo/Documents/tsalo/NiMARE/nimare/tests/data/nidm_pain_dset.json'

In [3]:
ddict = {}
folders = sorted(glob('/Users/tsalo/Downloads/nidm-pain-results/pain_*.nidm'))
for folder in folders:
    name = basename(folder)
    ddict[name] = {}
    ddict[name]['contrasts'] = {}
    ddict[name]['contrasts']['1'] = {}
    ddict[name]['contrasts']['1']['coords'] = {}
    ddict[name]['contrasts']['1']['coords']['space'] = 'MNI'
    ddict[name]['contrasts']['1']['images'] = {}
    ddict[name]['contrasts']['1']['images']['space'] = 'MNI_2mm'
    # con file
    files = glob(join(folder, 'Contrast*.nii.gz'))
    files = [f for f in files if 'StandardError' not in basename(f)]
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['con'] = f
    # se file
    files = glob(join(folder, 'ContrastStandardError*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['se'] = f
    # z file
    files = glob(join(folder, 'ZStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['z'] = f
    # t file
    # z file
    files = glob(join(folder, 'TStatistic*.nii.gz'))
    if files:
        f = sorted(files)[0]
    else:
        f = None
    ddict[name]['contrasts']['1']['images']['t'] = f
    # sample size
    f = join(folder, 'DesignMatrix.csv')
    if isfile(f):
        df = pd.read_csv(f, header=None)
        n = [df.shape[0]]
    else:
        n = None
    ddict[name]['contrasts']['1']['sample_sizes'] = n
    # foci
    files = glob(join(folder, 'ExcursionSet*.nii.gz'))
    f = sorted(files)[0]
    img = nib.load(f)
    data = np.nan_to_num(img.get_data())
    # positive clusters
    binarized = np.copy(data)
    binarized[binarized>0] = 1
    binarized[binarized<0] = 0
    binarized = binarized.astype(int)
    labeled = meas.label(binarized, np.ones((3, 3, 3)))[0]
    clust_ids = sorted(list(np.unique(labeled)[1:]))
    ijk = np.hstack([np.where(data * (labeled == c) == np.max(data * (labeled == c))) for c in clust_ids])
    ijk = ijk.T
    xyz = nib.affines.apply_affine(img.affine, ijk)
    ddict[name]['contrasts']['1']['coords']['x'] = list(xyz[:, 0])
    ddict[name]['contrasts']['1']['coords']['y'] = list(xyz[:, 1])
    ddict[name]['contrasts']['1']['coords']['z'] = list(xyz[:, 2])
    

In [4]:
with open(f2, 'w') as fo:
    json.dump(ddict, fo, sort_keys=True, indent=4)